In [1]:
import numpy as np
import tensorflow as tf
import keras as k
from keras.layers import BatchNormalization, Input, Conv2D, Activation, Add, Conv2DTranspose, Deconv2D
from keras.initializers import glorot_uniform
from keras.models import Model, Sequential
from keras.regularizers import l2

/Users/yichen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
weight_decay = 0
modules = 2
blocks_per_module = 9
def test_block(inputshape = (500, 500, 3)):
    inputimg = Input(shape=inputshape)
    x = Conv2D(32,(3,3),padding='SAME',activation='relu',use_bias=False,kernel_regularizer=l2(weight_decay))(inputimg)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    for i in range(modules):
        for j in range(blocks_per_module):
            t = x
            if j == 0:
                if i == 0:
                    print(i)
                    t = Conv2D(int(32*(2**i)),(3,3),padding='SAME',name = 'first_time', use_bias=False,kernel_regularizer=l2(weight_decay))(t)
                else:
                    t = Conv2D(int(32*(2**i)),(3,3),strides=(2,2),padding='SAME',use_bias=False,kernel_regularizer=l2(weight_decay))(t)
                    x = Conv2D(int(32*(2**i)),(1,1),strides=(2,2),padding='SAME',use_bias=False,kernel_regularizer=l2(weight_decay))(x)
            t = Conv2D(int(32*(2**i)),(3,3),padding='SAME',use_bias=False,kernel_regularizer=l2(weight_decay))(t)
            t = BatchNormalization()(t)
            t = Activation('relu')(t)
            print(t.shape)
            x = Add()([x,t])
            x = Activation('relu')(x)
            print('x.shape: {}'.format(x.shape))
    x = BatchNormalization()(x)
    outputs = Activation('softmax')(x)
    model = Model(inputs=inputimg, outputs=outputs)
    print(model.summary())
            

In [3]:
test_block()

0
(?, 500, 500, 32)
x.shape: (?, 500, 500, 32)
(?, 500, 500, 32)
x.shape: (?, 500, 500, 32)
(?, 500, 500, 32)
x.shape: (?, 500, 500, 32)
(?, 500, 500, 32)
x.shape: (?, 500, 500, 32)
(?, 500, 500, 32)
x.shape: (?, 500, 500, 32)
(?, 500, 500, 32)
x.shape: (?, 500, 500, 32)
(?, 500, 500, 32)
x.shape: (?, 500, 500, 32)
(?, 500, 500, 32)
x.shape: (?, 500, 500, 32)
(?, 500, 500, 32)
x.shape: (?, 500, 500, 32)
(?, 250, 250, 64)
x.shape: (?, 250, 250, 64)
(?, 250, 250, 64)
x.shape: (?, 250, 250, 64)
(?, 250, 250, 64)
x.shape: (?, 250, 250, 64)
(?, 250, 250, 64)
x.shape: (?, 250, 250, 64)
(?, 250, 250, 64)
x.shape: (?, 250, 250, 64)
(?, 250, 250, 64)
x.shape: (?, 250, 250, 64)
(?, 250, 250, 64)
x.shape: (?, 250, 250, 64)
(?, 250, 250, 64)
x.shape: (?, 250, 250, 64)
(?, 250, 250, 64)
x.shape: (?, 250, 250, 64)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to            

In [4]:
def residual_block(X, f, stage, id_block = True):
    X_shortcut = X
    
    conv_name = 'conv_' + 'filters_' + str(f) + 'stage_' + str(stage)
    bn_base_name = 'batch_normal' + 'filters_' + str(f) + 'stage_' + str(stage)

    X = Conv2D(filters = f, padding='same', kernel_size=(3,3),
              name = conv_name + '_b')(X)
    X = BatchNormalization(name = bn_base_name + '_base')(X)
    X = Activation('relu')(X)
    if id_block:
        X = Conv2D(filters = f, padding='same', kernel_size=(3,3),
               name = conv_name + '_idblock_a')(X)
    else:
        X = Conv2D(filters = f, padding='same', kernel_size=(3,3), strides = (2,2),
               name = conv_name + '_idblock_a')(X)
        X_shortcut = Conv2D(filters=f, strides=(2,2), padding='same', 
                        kernel_size=(1,1), name = conv_name + '_shortcut')(X_shortcut)
        X_shortcut = BatchNormalization(name = bn_base_name + '_shortcut')(X_shortcut)
        
    X = BatchNormalization()(X)
    X = Add()([X, X_shortcut])
    X = BatchNormalization(name = bn_base_name + '_shortcut_merge')(X)
    X = Activation('relu')(X)
    
    return X

In [5]:
def low_level_residual_blocks(X, input_shape):#input_shape = (500, 500, 3)):
    filter_list = [32, 64]
    #inputs = Input(input_shape)
    X = Conv2D(filters = 32, padding='same', kernel_size=(3,3), input_shape = input_shape,
               name = 'first_conv2d')(X)
    X = BatchNormalization(name = 'first_batch')(X)
    X = Activation('relu')(X)
    for i in range(1, 10):
        X = residual_block(X, 32, i)
    for i in range(1, 10):
        if i == 1:
            X = residual_block(X, 64, i, False)
        else:
            X = residual_block(X, 64, i)
    return X

In [6]:
def module3_divergent_residual_block(input_shape = (500, 500, 3)):
    input_img = Input(input_shape)
    
    X = low_level_residual_blocks(input_img, input_shape)
    
    #first divergent path, 1*1 conv layer
    X_diverge_one = Conv2D(filters = 2, kernel_size=(1,1), padding = 'same',
                           name = 'Conv2D_diverge_one')(X)
    X_diverge_one = BatchNormalization(name='bn_diverge_one')(X_diverge_one)
    X_diverge_one = Activation('relu')(X_diverge_one)
    
    for i in range(1, 10):
        if i == 1:
            X = residual_block(X, 128, i, False)
        else:
            X = residual_block(X, 128, i)
    #this parts find the conditional probability for further OPI operation
    
    #X_diverage_two = X # second divergent path
    
    #second divergent path, stack one conv layer and one deconv layer
    X_diverge_two = Conv2D(filters = 2, kernel_size=(1,1), padding = 'same',
                          name = 'Conv2D_diverge_two')(X)
    X_diverge_two = BatchNormalization(name='bn_conv_diverge_two')(X_diverge_two)
    X_diverge_two = Activation('relu')(X_diverge_two)
    print('shape before deconv2D: {}'.format(X_diverge_two))
    X_diverge_two = Conv2DTranspose(filters=2, kernel_size=(3,3),strides=(2,2), padding = 'same',
                                   name = 'Deconv_before_summation')(X_diverge_two)
    X_diverge_two = Activation('relu')(X_diverge_two)
    #Element-wise summation
    print('X_diverge_one.shape: {}, \
          X_diverge_two.shape: {}'.format(X_diverge_one.shape, X_diverge_two.shape))
    X_merge = Add()([X_diverge_two, X_diverge_one])
    #X_merge = BatchNormalization()(X_merge)
    #X_merge = Activation('relu')(X_merge)
    X_merge = Conv2DTranspose(filters=2, kernel_size=(3,3), strides=(2,2), padding='same',
                             name = 'Deconv_detection_final_layer')(X_merge)
    X_merge = BatchNormalization()(X_merge)
    outputs = Activation('softmax')(X_merge)
    print('this is temp outputs.shape:'.format(outputs.shape))
    model = Model(inputs=input_img, outputs = outputs)
    print(model.summary())

In [7]:
module3_divergent_residual_block()

shape before deconv2D: Tensor("activation_95/Relu:0", shape=(?, 125, 125, 2), dtype=float32)
X_diverge_one.shape: (?, 250, 250, 2),           X_diverge_two.shape: (?, ?, ?, 2)
this is temp outputs.shape:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 500, 500, 3)  0                                            
__________________________________________________________________________________________________
first_conv2d (Conv2D)           (None, 500, 500, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
first_batch (BatchNormalization (None, 500, 500, 32) 128         first_conv2d[0][0]               
__________________________________________________________________________________________________
acti

In [204]:
def fine_grained_block(X):
    print('fine_grained input X.shape: {}'.format(X.shape))
    for i in range(1, 10):
        X = residual_block(X, 128, i)
    X = Conv2D(kernel_size=(1, 1), filters=5, padding='same', name = 'fine_grained_conv')
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Conv2DTranspose(strides=(2,2), filters=5, kernel_size = (3,3), padding = 'same', 
                        name = 'fine_grained_deconv_1')
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Conv2DTranspose(strides=(2,2), filters=5, kernel_size = (3,3), padding = 'same', 
                        name = 'fine_grained_deconv_1')
    X = BatchNormalization()(X)
    X = Activation('softmax')(X)
    print('fine_grained output X.shape: {}'.format(X.shape)
    return X

In [220]:
import scipy
from scipy.io import loadmat, savemat

In [225]:
path = '/Users/yichen//Desktop/CRCHistoPhenotypes_2016_04_28/Detection/'
for i in range(1, 100):
    img_path = path + 'img'+str(i) + '/img' + str(i) + '_detection.mat'
    a = loadmat(img_path)
a = loadmat(path + 'img'+str(1) + '/img' + str(1) + '_detection.mat')
a

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Wed Dec  9 17:36:28 2015',
 '__version__': '1.0',
 'detection': array([[148.78269485, 181.47754293],
        [ 96.60303831, 159.68097754],
        [ 79.42998679, 172.89101717],
        [171.9002642 ,  71.834214  ],
        [ 95.77741083, 116.08784676],
        [410.34147952,  33.19484808],
        [419.91875826,  46.73513871],
        [172.89101717, 197.16446499],
        [168.2675033 , 204.42998679],
        [110.96895641, 383.42602378],
        [108.65719947, 424.70739762],
        [  8.52425107,  15.65691869],
        [ 38.48145506,  39.90798859],
        [ 22.78958631,  50.25035663],
        [ 69.86519258,  72.00499287],
        [328.76948481,  60.60568032],
        [327.77873184,  79.42998679],
        [108.65719947, 446.17371202],
        [219.12615588, 251.32562748],
        [111.62945839, 462.35601057],
        [ 93.90846995, 177.41256831],
        [169.72814208, 117.64480874],
        [ 96

In [228]:
def detection_error(N, alpha, true_label, p_obj, p_bkg):
    first_part = np.negative(np.divide(1, N))
    summation_part = 0
    summation = 0
    for i in range(1, N):
        if true_label[i] == 0: #location being a background
            summation_part = np.log(p_bkg[i])
        elif true_label[i] == 1: #location being a nuclei
            summation_part = np.multiply(alpha, np.log(p_obj[i]))
        summation = np.add(summation, summation_part)
    L_det = np.multiply(first_part, summation)
    return L_det

In [ ]:
def classification_error(threshold, p_obj):
    threshold = 0.8
    

In [ ]:
def total_error(lambd, beta, )

In [94]:
a = tf.Variable(tf.random_uniform([500, 500, 3]))
b = tf.Variable(tf.random_uniform([500, 500, 3]))
c = Add()([a,b])
c

<tf.Tensor 'add_425_1/add:0' shape=(500, 500, 3) dtype=float32>